In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [4]:
from dnaseq.constants import *
from dnaseq.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        
        return data_ingestion_config

In [5]:
import os
import urllib.request as request
from dnaseq import logger
from dnaseq.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with folowing info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

[2024-03-27 20:47:37,090: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-27 20:47:37,095: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 20:47:37,099: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-27 20:47:37,101: INFO: common: created directory at: artifacts]
[2024-03-27 20:47:37,102: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-27 20:47:39,090: INFO: 266555842: artifacts/data_ingestion/human_data.txt downloaded with folowing info: 
Connection: close
Content-Length: 5552098
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "75b6a98a7d3638ed7a5b842f1918768496940efcb3a3dc64dbc2806c87da8966"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7554:2F9663:19B131C:1ABFC5A:660477D9
Accept-Ranges: 